In [1]:
#Import packages
#import numpy as np
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import  Normalizer, LemmatizerModel, StopWordsCleaner, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, VectorAssembler, HashingTF, IDF, Tokenizer
#from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1601581867580_0001,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#Snowflake connection info
sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : 'APP_SPARK_PROD',
  'sfPassword' : '>6txhcQ*7&^Qrq`j',
  'sfDatabase' : 'ANALYTICS_PROD',
  'sfSchema' : 'ANALYTICS',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
query_old = "select * \
from \
(select ugc_id as mod_ugc_id, created_date as mod_date, year(created_date)*100+month(created_date) as YYYYMM \
from cdm.observation as o \
where observation_type_code = 'PM' and observation_client_id =1 and ugc_type = 'review') as moderated_reviews \
left join \
(select ugc_id, observation_type_code \
from cdm.observation as o \
where observation_type_code in ('NR','PII','PR','CS','DC','CR','FRD','TST','CC','EWL','SC','PIM','LC','WEB','FL','SA','URL','CV','US') \
 and observation_client_id =1 and ugc_type = 'review') as observation_reviews \
on moderated_reviews.mod_ugc_id = observation_reviews.ugc_id \
left join \
(select ugc_id as r_ugc_id, review_headline, review_comments, review_rating, reviewer_nickname, locale \
from analytics.review ) as r \
on r.r_ugc_id = moderated_reviews.mod_ugc_id \
where locale in ('en_US','en_GB','en_CA') \
and mod_date > '2020-09-21'   "



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
query = "select distinct r.ugc_id, r.created_date, r.review_headline, \
r.review_comments, r.review_rating, r.reviewer_nickname, r.locale, r.observations, \
case when contains(r.observations, 'PM') then 1 else 0 end as PM, \
case when contains(r.observations, 'NR') then 1 else 0 end as NR, \
case when contains(r.observations, 'PII') then 1 else 0 end as PII, \
case when contains(r.observations, 'PR') then 1 else 0 end as PR, \
case when contains(r.observations, 'CS') then 1 else 0 end as CS, \
case when contains(r.observations, 'DC') then 1 else 0 end as DC, \
case when contains(r.observations, 'CR') then 1 else 0 end as CR, \
case when contains(r.observations, 'FRD') then 1 else 0 end as FRD, \
case when contains(r.observations, 'TST') then 1 else 0 end as TST, \
case when contains(r.observations, 'CC') then 1 else 0 end as CC, \
case when contains(r.observations, 'SC') then 1 else 0 end as SC, \
case when contains(r.observations, 'PIM') then 1 else 0 end as PIM, \
case when contains(r.observations, 'LC') then 1 else 0 end as LC, \
case when contains(r.observations, 'WEB') then 1 else 0 end as WEB, \
case when contains(r.observations, 'FL') then 1 else 0 end as FL, \
case when contains(r.observations, 'SA') then 1 else 0 end as SA, \
case when contains(r.observations, 'URL') then 1 else 0 end as URL, \
case when contains(r.observations, 'CV') then 1 else 0 end as CV, \
case when contains(r.observations, 'US') then 1 else 0 end as US, \
case when r.observations = 'PM' then 1 else 0 end as NOLABEL \
from analytics.review as r \
left join cdm.observation as o \
on r.ugc_id = o.ugc_id \
where contains(r.observations, 'PM') \
and r.locale in ('en_US','en_GB','en_CA') \
and o.created_date > '2020-09-23'  \
limit 100"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
query_testing = "select distinct r.ugc_id, r.created_date, r.review_headline, \
r.review_comments, r.review_rating, r.reviewer_nickname, r.reviewer_location, r.observations, \
case when contains(r.observations, 'PM') then 1 else 0 end as PM, \
case when contains(r.observations, 'NR') then 1 else 0 end as NR, \
case when contains(r.observations, 'PII') then 1 else 0 end as PII, \
case when contains(r.observations, 'PR') then 1 else 0 end as PR, \
case when contains(r.observations, 'CS') then 1 else 0 end as CS, \
case when contains(r.observations, 'DC') then 1 else 0 end as DC, \
case when contains(r.observations, 'CR') then 1 else 0 end as CR, \
case when contains(r.observations, 'FRD') then 1 else 0 end as FRD, \
case when contains(r.observations, 'TST') then 1 else 0 end as TST, \
case when contains(r.observations, 'CC') then 1 else 0 end as CC, \
case when contains(r.observations, 'SC') then 1 else 0 end as SC, \
case when contains(r.observations, 'PIM') then 1 else 0 end as PIM, \
case when contains(r.observations, 'LC') then 1 else 0 end as LC, \
case when contains(r.observations, 'WEB') then 1 else 0 end as WEB, \
case when contains(r.observations, 'FL') then 1 else 0 end as FL, \
case when contains(r.observations, 'SA') then 1 else 0 end as SA, \
case when contains(r.observations, 'URL') then 1 else 0 end as URL, \
case when contains(r.observations, 'CV') then 1 else 0 end as CV, \
case when contains(r.observations, 'US') then 1 else 0 end as US, \
case when r.observations = 'PM' then 1 else 0 end as NOLABEL \
from analytics.review as r \
where r.locale in ('en_US','en_GB','en_CA') \
and r.created_date > '2018-09-01'  \
and contains(r.observations, 'PM') \
limit 100000"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Use review data set to get observation codes bc they are already comma separated and less joins to perform
#Still need observation set though to get observation created date
#This is bc review could be from ages ago but we could have imported recently and moderated it
#Want to make sure we include as many observation labels as possible for most accurate model



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#Spark data frame

reviews = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
          .options(**sfOptions).option("query", query_testing).load()
reviews.show(5)
reviews.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+-------------+-----------------+-----------------+------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+
|   UGC_ID|        CREATED_DATE|     REVIEW_HEADLINE|     REVIEW_COMMENTS|REVIEW_RATING|REVIEWER_NICKNAME|REVIEWER_LOCATION|OBSERVATIONS| PM| NR|PII| PR| CS| DC| CR|FRD|TST| CC| SC|PIM| LC|WEB| FL| SA|URL| CV| US|NOLABEL|
+---------+--------------------+--------------------+--------------------+-------------+-----------------+-----------------+------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+
|234199352|2019-01-29 00:00:...|         Comfy shoes|My husband loves ...|            5|          Giselle|      San Juan PR|       CM,PM|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|      0|
|234259318|2019-01-29 00:00:...|      Yes yes yes!!!|I saw that kids s...|            5|      Jenni jenni|      

In [24]:
#Change locations with just 'US' to 'USA' otherwise the lemmatizer will change 'us' to 'we'
reviews = reviews.withColumn('LOCATION_UPDATED', F.when(F.col('REVIEWER_LOCATION')=='US', 'USA').otherwise(F.col('REVIEWER_LOCATION'))) \
                    .withColumn('ALL_TEXT', F.concat(F.lit('headline '),'REVIEW_HEADLINE', F.lit(' comment '), 'REVIEW_COMMENTS', \
                             F.lit(' nickname '),'REVIEWER_NICKNAME', F.lit(' location '), 'LOCATION_UPDATED')) \
                    .drop('REVIEW_HEADLINE','REVIEW_COMMENTS','REVIEWER_NICKNAME','LOCATION_UPDATED','REVIEWER_LOCATION')
#reviews.select('REVIEW_HEADLINE','REVIEW_COMMENTS','ALL_TEXT').show(truncate=False)
reviews.select('ALL_TEXT').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ALL_TEXT                                                                                                                           

In [25]:
#Look for text that contains specific price mention (9.99 = yes, 20 = no)
reviews_2 = reviews.withColumn('PRICE', F.when(F.col('ALL_TEXT').rlike('\d+(?:[.]\d{2})')==False,0).otherwise(1))
reviews_2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------------+------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+--------------------+-----+
|   UGC_ID|       CREATED_DATE|REVIEW_RATING|OBSERVATIONS| PM| NR|PII| PR| CS| DC| CR|FRD|TST| CC| SC|PIM| LC|WEB| FL| SA|URL| CV| US|NOLABEL|            ALL_TEXT|PRICE|
+---------+-------------------+-------------+------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+--------------------+-----+
|233995151|2019-01-29 00:00:00|            5|       CM,PM|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|      0|headline I live t...|    0|
|234033292|2019-01-29 00:00:00|            5|       CM,PM|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|      0|headline Love lov...|    0|
|234134471|2019-01-29 00:00:00|            5|       CM,PM|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|      0|headline

In [26]:
#spark-nlp package can only process after text has been turned into a "document"
documentAssembler = DocumentAssembler().setInputCol('ALL_TEXT') .setOutputCol('DOCUMENT_RAW')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#If this gives setInputCols error - run import of Tokenizer again
from sparknlp.annotator import  Tokenizer
tokenizer = Tokenizer().setInputCols(['DOCUMENT_RAW']).setOutputCol('TOKEN')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#print(stopwords.words('english'))
stop_words = stopwords.words('english')
stopwords_cleaner = StopWordsCleaner().setInputCols(['TOKEN']).setOutputCol('TOKEN_NOSTOPWORDS').setStopWords(stop_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#Normalizing removes punctuation like commas and periods, good to have
#But also removes @ from @ss, or * from f*ck leaving @ss as 'ss' and f*ck as 'fck'
#http://www.amazon.com becomes httpwwwamazoncom
#But as long as this is consistent then model should still learn to associate 'ss' and 'fck' as profanity

#Normalizing completely removes $9.99 and price references, removes emojis
#NOT good for price label or for profanity label

#setCleanupPatterns(patterns): Regular expressions list for normalization, defaults [^A-Za-z]
#.setCleanupPatterns(["[^\w\d\s]"]).  removes not word, not  digit, not white space
#try removing only ,.!-'  but keep $*:/        regex: [\,\.\!\-\']
#This will remove normal sentence ending punct and commas but KEEPS @ss and f*ck and http://wwwamazoncom and $999 and emojis
normalizer = Normalizer().setInputCols(['TOKEN_NOSTOPWORDS']).setOutputCol('NORMALIZED').setLowercase(True) \
  .setCleanupPatterns(["[\,\.\!\-\'\(\)\?]"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
lemmatizer = LemmatizerModel.pretrained().setInputCols(['NORMALIZED']).setOutputCol('LEMMA')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [31]:
#Finisher - puts transformed columns into human readable form
finisher = Finisher().setInputCols(['TOKEN','TOKEN_NOSTOPWORDS','NORMALIZED','LEMMA']).setCleanAnnotations(False)
pipeline = Pipeline().setStages([documentAssembler, tokenizer, stopwords_cleaner, normalizer, lemmatizer, finisher])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
#Run reviews through cleaning pipeline
reviews_3 = pipeline.fit(reviews_2).transform(reviews_2)
reviews_4 = reviews_3.drop('DOCUMENT_RAW','TOKEN','TOKEN_NOSTOPWORDS','NORMALIZED','LEMMA')
reviews_4.show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------------+------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+--------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|UGC_ID   |CREATED_DATE       |REVIEW_RATING|OBSERVATIONS|PM |NR |PII|PR |CS |DC |CR |FRD|TST|CC |SC |PIM|LC |WEB|FL |SA |URL|CV |US |NOLABEL|ALL_TEXT                                                                              |PRICE|finished_TOKEN                                                                                         |finished_TOKEN_NOSTOPWORDS                                 

In [48]:
reviews_4.explain(extended=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
Project [UGC_ID#1002, CREATED_DATE#1003, REVIEW_RATING#1006, OBSERVATIONS#1009, PM#1010, NR#1011, PII#1012, PR#1013, CS#1014, DC#1015, CR#1016, FRD#1017, TST#1018, CC#1019, SC#1020, PIM#1021, LC#1022, WEB#1023, FL#1024, SA#1025, URL#1026, CV#1027, US#1028, NOLABEL#1029, ... 6 more fields]
+- Project [UGC_ID#1002, CREATED_DATE#1003, REVIEW_RATING#1006, OBSERVATIONS#1009, PM#1010, NR#1011, PII#1012, PR#1013, CS#1014, DC#1015, CR#1016, FRD#1017, TST#1018, CC#1019, SC#1020, PIM#1021, LC#1022, WEB#1023, FL#1024, SA#1025, URL#1026, CV#1027, US#1028, NOLABEL#1029, ... 11 more fields]
   +- Project [UGC_ID#1002, CREATED_DATE#1003, REVIEW_RATING#1006, OBSERVATIONS#1009, PM#1010, NR#1011, PII#1012, PR#1013, CS#1014, DC#1015, CR#1016, FRD#1017, TST#1018, CC#1019, SC#1020, PIM#1021, LC#1022, WEB#1023, FL#1024, SA#1025, URL#1026, CV#1027, US#1028, NOLABEL#1029, ... 10 more fields]
      +- Project [UGC_ID#1002, CREATED_DATE#1003, REVIEW_RATING#1006, OBSERVATIONS#1009, PM#1

In [53]:
#sc = spark.sparkContext
#sc.setCheckpointDir('checkpoint')
#reviews_4.checkpoint()
reviews_4.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o2158.checkpoint.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 21.0 failed 4 times, most recent failure: Lost task 1.3 in stage 21.0 (TID 95, ip-10-101-215-75.us-west-2.compute.internal, executor 2): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:585)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1862)
	at org.apache.spark.rdd

In [49]:
reviews_5 = spark.createDataFrame(reviews_4.rdd, schema=reviews_4.schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
reviews_5.explain(extended=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
LogicalRDD [UGC_ID#4743, CREATED_DATE#4744, REVIEW_RATING#4745, OBSERVATIONS#4746, PM#4747, NR#4748, PII#4749, PR#4750, CS#4751, DC#4752, CR#4753, FRD#4754, TST#4755, CC#4756, SC#4757, PIM#4758, LC#4759, WEB#4760, FL#4761, SA#4762, URL#4763, CV#4764, US#4765, NOLABEL#4766, ... 6 more fields], false

== Analyzed Logical Plan ==
UGC_ID: decimal(38,0), CREATED_DATE: timestamp, REVIEW_RATING: decimal(38,0), OBSERVATIONS: string, PM: decimal(1,0), NR: decimal(1,0), PII: decimal(1,0), PR: decimal(1,0), CS: decimal(1,0), DC: decimal(1,0), CR: decimal(1,0), FRD: decimal(1,0), TST: decimal(1,0), CC: decimal(1,0), SC: decimal(1,0), PIM: decimal(1,0), LC: decimal(1,0), WEB: decimal(1,0), FL: decimal(1,0), SA: decimal(1,0), URL: decimal(1,0), CV: decimal(1,0), US: decimal(1,0), NOLABEL: decimal(1,0), ... 6 more fields
LogicalRDD [UGC_ID#4743, CREATED_DATE#4744, REVIEW_RATING#4745, OBSERVATIONS#4746, PM#4747, NR#4748, PII#4749, PR#4750, CS#4751, DC#4752, CR#4753, FRD#4754,

In [51]:
reviews_5.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[UGC_ID: decimal(38,0), CREATED_DATE: timestamp, REVIEW_RATING: decimal(38,0), OBSERVATIONS: string, PM: decimal(1,0), NR: decimal(1,0), PII: decimal(1,0), PR: decimal(1,0), CS: decimal(1,0), DC: decimal(1,0), CR: decimal(1,0), FRD: decimal(1,0), TST: decimal(1,0), CC: decimal(1,0), SC: decimal(1,0), PIM: decimal(1,0), LC: decimal(1,0), WEB: decimal(1,0), FL: decimal(1,0), SA: decimal(1,0), URL: decimal(1,0), CV: decimal(1,0), US: decimal(1,0), NOLABEL: decimal(1,0), ALL_TEXT: string, PRICE: int, finished_TOKEN: array<string>, finished_TOKEN_NOSTOPWORDS: array<string>, finished_NORMALIZED: array<string>, finished_LEMMA: array<string>]

In [54]:
#WORKS FINE ON SMALL DATA SETS BUT ERRORS ON LARGE ONES
#m5a.2xlarge master + m5a.2xlarge node: 30k rows works with cache, 60k with cache DOESN'T, 
#60k with change to rdd - works (10 features = 130 secs, 1000 features = 128 secs) 
#100k rows - rdd - 1000 features - error
#m5a.2xlarge master + m5a.2xlarge FIVE nodes: 100k - rdd - error, 80k - rdd - error
#m5.12xlarge master + m5.12xlarge node: 200k - rdd - error, 100k - rdd - error, 100k - cache - fail, 100k - rdd and cache - fail, 100k - checkpoint 

#An error occurred while calling o2069.fit.
#: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 4 times, most recent failure: Lost task 0.3 in stage 33.0 (TID 59, ip-10-101-214-119.us-west-2.compute.internal, executor 6): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>)

#TF-IDF

hashingTF = HashingTF(inputCol='finished_LEMMA', outputCol='RAW_TFIDF_FEATURES', numFeatures=1000)
featurized_data = hashingTF.transform(reviews_4)
#featurized_data.show()
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol='RAW_TFIDF_FEATURES', outputCol='TFIDF_FEATURES')
idf_model = idf.fit(featurized_data)
#idf_model.show()
tfidf_results = idf_model.transform(featurized_data)

tfidf_results.select('finished_LEMMA','TFIDF_FEATURES').show(truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o2310.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 22.0 failed 4 times, most recent failure: Lost task 1.3 in stage 22.0 (TID 108, ip-10-101-215-75.us-west-2.compute.internal, executor 9): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:585)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$

In [61]:
#Append additional numeric columns to existing TF-IDF features vector
vectorAssembler = VectorAssembler(inputCols = ['TFIDF_FEATURES','REVIEW_RATING','PRICE'], outputCol = 'features')
test = vectorAssembler.transform(tfidf_results)
test.select('ALL_TEXT','TFIDF_FEATURES','REVIEW_RATING','PRICE','features').show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------+-------------+-----+----------------------------------------------------------------------------------------------------+
|ALL_TEXT                                                                                                                          |TFIDF_FEATURES                                                                                                 |REVIEW_RATING|PRICE|features                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------+-------------+-----+----

In [49]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat"),
    (0.0, "Logistic regression models are dumb")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[0,5,9,17],[0...|
|  0.0|(20,[2,7,9,13,15]...|
|  1.0|(20,[4,6,13,15,18...|
|  0.0|(20,[4,8,13,15,18...|
+-----+--------------------+

In [50]:
rescaledData.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----------------------------------+------------------------------------------+-----------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|label|sentence                           |words                                     |rawFeatures                              |features                                                                                                              |
+-----+-----------------------------------+------------------------------------------+-----------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |(20,[0,5,9,17],[1.0,1.0,1.0,2.0])        |(20,[0,5,9,17],[0.9162907318741551,0.9162907318741551,0.5108256237659907,1.8325814637483102])                         |
|0.0  |I

In [51]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = rescaledData

lr = LogisticRegression(maxIter=3, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coefficients: (20,[6],[0.6599674909832574])
Intercept: -1.1875233653399353

In [52]:
#Show what labels the model predicted
lrModel_2 = lrModel.transform(training)
lrModel_2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|            sentence|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(20,[0,5,9,17],[1...|(20,[0,5,9,17],[0...|[1.18752336533993...|[0.76629782750481...|       0.0|
|  0.0|I wish Java could...|[i, wish, java, c...|(20,[2,7,9,13,15]...|(20,[2,7,9,13,15]...|[1.18752336533993...|[0.76629782750481...|       0.0|
|  1.0|Logistic regressi...|[logistic, regres...|(20,[4,6,13,15,18...|(20,[4,6,13,15,18...|[0.58280127001373...|[0.64171172349340...|       0.0|
|  0.0|Logistic regressi...|[logistic, regres...|(20,[4,8,13,15,18...|(20,[4,8,13,15,18...|[1.18752336533993...|[0.76629782750481.

In [53]:

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

objectiveHistory:
0.5623351446188085
0.5501013689318484
0.5421128362877067
0.5315519433187645

In [54]:
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False positive rate by label:
label 0: 1.0
label 1: 0.0
True positive rate by label:
label 0: 1.0
label 1: 0.0

In [55]:
print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Precision by label:
label 0: 0.75
label 1: 0.0
Recall by label:
label 0: 1.0
label 1: 0.0
F-measure by label:
label 0: 0.8571428571428571
label 1: 0.0

In [56]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.75
FPR: 0.75
TPR: 0.75
F-measure: 0.6428571428571428
Precision: 0.5625
Recall: 0.75

In [ ]:
#Train test split - use this over cross fold since it will run quicker even though potentially less accurate results
#Large enough data set should account for not using cross fold

train, test = reviews.randomSplit([0.9, 0.1], seed=12345)
train.show()
train.count()
#884/1000 = 88%